# This is the Jupyter Notebook for the MAGICODE project

### First install some modules that might not be installed

In [ ]:
# Install a pip package in the current Jupyter kernel
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install tensorflow
# install sklearn to use train_test_split function
!{sys.executable} -m pip install sklearn
# install opencv to use cv2 module in get_preprocessed_img
!{sys.executable} -m pip install opencv-python
# install pydot to use keras plot_model function
# !{sys.executable} -m pip install pydot
# install graphviz to use keras plot_model function
# !{sys.executable} -m pip install graphviz

### Then import some libraries and modules that are needed for the code to run

In [ ]:
import os
import sklearn.model_selection as model_selection
import glob
import shutil
import datetime
import pydot
import numpy as np
from pathlib import Path
from os.path import join

### Define some values used later

In [ ]:
SOURCE = 'dataset'
TRAINING_SET_NAME = 'training_set'
EVALUATION_SET_NAME = 'eval_set'
TRAINING_FEATURES = 'training_features'
EVAL_FEATURES = 'eval_features'
START_TOKEN = "<START>"
END_TOKEN = "<END>"
PLACEHOLDER = " "
SEPARATOR = '->'
LOG_DIR = join('logs', 'fit' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
CHECKPOINT_DIR = 'training'
CHECKPOINT_FILE_NAME = 'weights_checkpoint'
CHECKPOINT_FILE = join(CHECKPOINT_FILE_NAME + '.hdf5')
CHECKPOINT_ARCHIVE = join(CHECKPOINT_FILE_NAME + '.zip')
CHECKPOINT_PATH = join(CHECKPOINT_DIR, CHECKPOINT_FILE)
CHECKPOINT_ARCHIVE_PATH = join(CHECKPOINT_DIR, CHECKPOINT_ARCHIVE)
# ================================ HYPERPARAMETERS ================================
EPOCHS = 30 # Using early stopping, so training might run for less than X epochs
IMAGE_SIZE = 256
BATCH_SIZE = 64
CONTEXT_LENGTH = 48
PATIENCE = 5
LEARNING_RATE = 0.1 # Using ReduceLROnPlateau, so learning rate gets lowered by 0.1 factor if no val_loss improvement
CLIPVALUE = 1.0

### Unzip the dataset

In [ ]:
# join the volumes together into a single zip file
!zip -s 0 dataset.zip -O dataset_joined.zip
# unzip the newly assembled archive into the current folder
!unzip dataset_joined.zip -d ./

if os.listdir(SOURCE):
    print ('files unzipped')

### Split dataset into training and evaluation sets

In [ ]:
# get all file paths
all_files = os.listdir(SOURCE)
# build a generic image path (e.g. 'all_data/dataset/*.png')
images_path = join(SOURCE, '*.png')
# get all images paths
img_files = glob.glob(images_path)
# remove files extension from files paths
img_files_without_extension = [Path(img_file).stem for img_file in img_files]

# splits randomly the files into two sets (train_set = 85% of dataset, eval_set = 15% of dataset)
train_set,eval_set = model_selection.train_test_split(img_files_without_extension, train_size=0.85)

# create the TRAINING_SET_NAME and EVALUATION_SET_NAME directories if they do not exist
if not os.path.exists(join(SOURCE, TRAINING_SET_NAME)):
    os.makedirs(join(SOURCE, TRAINING_SET_NAME))
if not os.path.exists(join(SOURCE, EVALUATION_SET_NAME)):
    os.makedirs(join(SOURCE, EVALUATION_SET_NAME))

# copy the files (img and gui) from the all_data folder into the training_set folder
for file in train_set:
    shutil.copyfile(join(SOURCE, file + '.png'), join(SOURCE, TRAINING_SET_NAME, file + '.png'))
    shutil.copyfile(join(SOURCE, file + '.gui'), join(SOURCE, TRAINING_SET_NAME, file + '.gui'))

# copy the files (img and gui) from the all_data folder into the eval_set folder
for file in eval_set:
    shutil.copyfile(join(SOURCE, file + '.png'), join(SOURCE, EVALUATION_SET_NAME, file + '.png'))
    shutil.copyfile(join(SOURCE, file + '.gui'), join(SOURCE, EVALUATION_SET_NAME, file + '.gui'))

print('Training dataset: {}'.format(join(SOURCE, TRAINING_SET_NAME)))
print('Evaluation dataset: {}'.format(join(SOURCE, EVALUATION_SET_NAME)))



### Define some Classes and functions that will be used a few times

In [ ]:
class Utils:
    @staticmethod
    def sparsify(label_vector, output_size):
        sparse_vector = []

        for label in label_vector:
            sparse_label = np.zeros(output_size)
            sparse_label[label] = 1

            sparse_vector.append(sparse_label)

        return np.array(sparse_vector)

    @staticmethod
    def get_preprocessed_img(img_path, image_size):
        import cv2
        img = cv2.imread(img_path)
        if not img is None:
            img = cv2.resize(img, (image_size, image_size))
            img = img.astype('float32')
            img /= 255
        return img
    
class Sampler:
    def __init__(self, voc_path, input_shape, output_size, context_length):
        self.voc = Vocabulary()
        self.voc.retrieve(voc_path)

        self.input_shape = input_shape
        self.output_size = output_size

        print('Vocabulary size: {}'.format(self.voc.size))
        print('Input shape: {}'.format(self.input_shape))
        print('Output size: {}'.format(self.output_size))

        self.context_length = context_length

    def predict_greedy(self, model, input_img, require_sparse_label=True, sequence_length=150, verbose=False):
        current_context = [self.voc.vocabulary[PLACEHOLDER]] * (self.context_length - 1)
        current_context.append(self.voc.vocabulary[START_TOKEN])
        if require_sparse_label:
            current_context = Utils.sparsify(current_context, self.output_size)

        predictions = START_TOKEN
        out_probas = []

        for i in range(0, sequence_length):
            if verbose:
                print('predicting {}/{}...'.format(i, sequence_length))

            probas = model.predict(input_img, np.array([current_context]))
            prediction = np.argmax(probas)
            out_probas.append(probas)

            new_context = []
            for j in range(1, self.context_length):
                new_context.append(current_context[j])

            if require_sparse_label:
                sparse_label = np.zeros(self.output_size)
                sparse_label[prediction] = 1
                new_context.append(sparse_label)
            else:
                new_context.append(prediction)

            current_context = new_context

            predictions += self.voc.token_lookup[prediction]

            if self.voc.token_lookup[prediction] == END_TOKEN:
                break

        return predictions, out_probas


class Vocabulary:
    def __init__(self):
        self.binary_vocabulary = {}
        self.vocabulary = {}
        self.token_lookup = {}
        self.size = 0

        self.append(START_TOKEN)
        self.append(END_TOKEN)
        self.append(PLACEHOLDER)

    def append(self, token):
        if token not in self.vocabulary:
            self.vocabulary[token] = self.size
            self.token_lookup[self.size] = token
            self.size += 1

    def create_binary_representation(self):
        if sys.version_info >= (3,):
            items = self.vocabulary.items()
        else:
            items = self.vocabulary.iteritems()
        for key, value in items:
            binary = np.zeros(self.size)
            binary[value] = 1
            self.binary_vocabulary[key] = binary

    def get_serialized_binary_representation(self):
        if len(self.binary_vocabulary) == 0:
            self.create_binary_representation()

        string = ''
        if sys.version_info >= (3,):
            items = self.binary_vocabulary.items()
        else:
            items = self.binary_vocabulary.iteritems()
        for key, value in items:
            array_as_string = np.array2string(value, separator=',', max_line_width=self.size * self.size)
            string += '{}{}{}\n'.format(key, SEPARATOR, array_as_string[1:len(array_as_string) - 1])
        return string

    def save(self, path):
        output_file_name = '{}/words.vocab'.format(path)
        output_file = open(output_file_name, 'w')
        output_file.write(self.get_serialized_binary_representation())
        output_file.close()

    def retrieve(self, path):
        input_file = open('{}/words.vocab'.format(path), 'r')
        buffer = ''
        for line in input_file:
            try:
                separator_position = len(buffer) + line.index(SEPARATOR)
                buffer += line
                key = buffer[:separator_position]
                value = buffer[separator_position + len(SEPARATOR):]
                value = np.fromstring(value, sep=',')

                self.binary_vocabulary[key] = value
                self.vocabulary[key] = np.where(value == 1)[0][0]
                self.token_lookup[np.where(value == 1)[0][0]] = key

                buffer = ''
            except ValueError:
                buffer += line
        input_file.close()
        self.size = len(self.vocabulary)

class Generator:
    @staticmethod
    def data_generator(voc, gui_paths, img_paths, batch_size, generate_binary_sequences=False, verbose=False, loop_only_one=False):
        assert len(gui_paths) == len(img_paths)
        voc.create_binary_representation()

        while 1:
            batch_input_images = []
            batch_partial_sequences = []
            batch_next_words = []
            sample_in_batch_counter = 0

            for i in range(0, len(gui_paths)):
                if img_paths[i].find('.png') != -1:
                    img = Utils.get_preprocessed_img(img_paths[i], IMAGE_SIZE)
                else:
                    img = np.load(img_paths[i])['features']
                gui = open(gui_paths[i], 'r')

                token_sequence = [START_TOKEN]
                for line in gui:
                    line = line.replace(',', ' ,').replace('\n', ' \n')
                    tokens = line.split(' ')
                    for token in tokens:
                        voc.append(token)
                        token_sequence.append(token)
                token_sequence.append(END_TOKEN)

                suffix = [PLACEHOLDER] * CONTEXT_LENGTH

                a = np.concatenate([suffix, token_sequence])
                for j in range(0, len(a) - CONTEXT_LENGTH):
                    context = a[j:j + CONTEXT_LENGTH]
                    label = a[j + CONTEXT_LENGTH]

                    batch_input_images.append(img)
                    batch_partial_sequences.append(context)
                    batch_next_words.append(label)
                    sample_in_batch_counter += 1

                    if sample_in_batch_counter == batch_size or (loop_only_one and i == len(gui_paths) - 1):
                        if verbose:
                            print('Generating sparse vectors...')
                        batch_next_words = Dataset.sparsify_labels(batch_next_words, voc)
                        if generate_binary_sequences:
                            batch_partial_sequences = Dataset.binarize(batch_partial_sequences, voc)
                        else:
                            batch_partial_sequences = Dataset.indexify(batch_partial_sequences, voc)

                        if verbose:
                            print('Convert arrays...')
                        batch_input_images = np.array(batch_input_images)
                        batch_partial_sequences = np.array(batch_partial_sequences)
                        batch_next_words = np.array(batch_next_words)

                        if verbose:
                            print('Yield batch')
                        yield ([batch_input_images, batch_partial_sequences], batch_next_words)

                        batch_input_images = []
                        batch_partial_sequences = []
                        batch_next_words = []
                        sample_in_batch_counter = 0
                        
class Dataset:
    def __init__(self):
        self.input_shape = None
        self.output_size = None

        self.ids = []
        self.input_images = []
        self.partial_sequences = []
        self.next_words = []

        self.voc = Vocabulary()
        self.size = 0

        self.dataset_name = ''

    @staticmethod
    def load_paths_only(path):
        print('Parsing data...')
        gui_paths = []
        img_paths = []
        for f in os.listdir(path):
            if f.find('.gui') != -1:
                path_gui = join(path, f)
                gui_paths.append(path_gui)
                file_name = f[:f.find('.gui')]

                if os.path.isfile(join(path, file_name + '.png')):
                    path_img = join(path, file_name + '.png')
                    img_paths.append(path_img)
                elif os.path.isfile(join(path, file_name + '.npz')):
                    path_img = join(path, file_name + '.npz')
                    img_paths.append(path_img)

        assert len(gui_paths) == len(img_paths)
        return gui_paths, img_paths

    def load(self, path, generate_binary_sequences=False):
        print('Loading {} data...'.format(self.dataset_name))
        for f in os.listdir(path):
            if f.find('.gui') != -1:
                gui = open(join(path, f), 'r')
                file_name = Path(f).stem
                if os.path.isfile(join(path, file_name + '.png')):
                    img = Utils.get_preprocessed_img(join(path, file_name + '.png'), IMAGE_SIZE)
                    self.append(file_name, gui, img)
                elif os.path.isfile(join(path, file_name + '.npz')):
                    img = np.load(join(path, file_name + '.npz'))['features']
                    self.append(file_name, gui, img)

        print('Generating sparse vectors...')
        self.voc.create_binary_representation()
        self.next_words = self.sparsify_labels(self.next_words, self.voc)
        if generate_binary_sequences:
            self.partial_sequences = self.binarize(self.partial_sequences, self.voc)
        else:
            self.partial_sequences = self.indexify(self.partial_sequences, self.voc)

        self.size = len(self.ids)
        assert self.size == len(self.input_images) == len(self.partial_sequences) == len(self.next_words)
        assert self.voc.size == len(self.voc.vocabulary)

        print('Dataset size: {}'.format(self.size))
        print('Vocabulary size: {}'.format(self.voc.size))

        self.input_shape = self.input_images[0].shape
        self.output_size = self.voc.size

        print('Input shape: {}'.format(self.input_shape))
        print('Output size: {}'.format(self.output_size))

    def convert_arrays(self):
        print('Convert arrays...')
        self.input_images = np.array(self.input_images)
        self.partial_sequences = np.array(self.partial_sequences)
        self.next_words = np.array(self.next_words)

    def append(self, sample_id, gui, img, to_show=False):
        if to_show:
            pic = img * 255
            pic = np.array(pic, dtype=np.uint8)
            Utils.show(pic)

        token_sequence = [START_TOKEN]
        for line in gui:
            line = line.replace(',', ' ,').replace('\n', ' \n')
            tokens = line.split(' ')
            for token in tokens:
                self.voc.append(token)
                token_sequence.append(token)
        token_sequence.append(END_TOKEN)

        suffix = [PLACEHOLDER] * CONTEXT_LENGTH

        a = np.concatenate([suffix, token_sequence])
        for j in range(0, len(a) - CONTEXT_LENGTH):
            context = a[j:j + CONTEXT_LENGTH]
            label = a[j + CONTEXT_LENGTH]

            self.ids.append(sample_id)
            self.input_images.append(img)
            self.partial_sequences.append(context)
            self.next_words.append(label)

    @staticmethod
    def indexify(partial_sequences, voc):
        temp = []
        for sequence in partial_sequences:
            sparse_vectors_sequence = []
            for token in sequence:
                sparse_vectors_sequence.append(voc.vocabulary[token])
            temp.append(np.array(sparse_vectors_sequence))

        return temp

    @staticmethod
    def binarize(partial_sequences, voc):
        temp = []
        for sequence in partial_sequences:
            sparse_vectors_sequence = []
            for token in sequence:
                sparse_vectors_sequence.append(voc.binary_vocabulary[token])
            temp.append(np.array(sparse_vectors_sequence))

        return temp

    @staticmethod
    def sparsify_labels(next_words, voc):
        temp = []
        for label in next_words:
            temp.append(voc.binary_vocabulary[label])

        return temp

    def save_metadata(self, path):
        np.save(join(path, 'meta_dataset'), np.array([self.input_shape, self.output_size, self.size], dtype=object), allow_pickle=True)

### Transform training set into numpy arrays

In [ ]:
#define source and destination folders
source = join(SOURCE, TRAINING_SET_NAME)
destination = join(SOURCE, TRAINING_FEATURES)

# create the training_features directory if it does not exist
if not os.path.exists(destination):
    os.makedirs(destination)

# transform images in training dataset (i.e. normalized pixel values and resized pictures) to numpy arrays (smaller files, useful if uploading the set to train a model in the cloud)
for f in os.listdir(source):
    if f.find('.png') != -1:
        img = Utils.get_preprocessed_img(join(source, f), IMAGE_SIZE)
        file_name = f[:f.find('.png')]

        np.savez_compressed(join(destination, file_name), features=img)
        retrieve = np.load(join(destination, file_name + '.npz'))['features']
        
        assert np.array_equal(img, retrieve)
        
        shutil.copyfile(join(source, file_name + '.gui'), join(destination, file_name + '.gui'))

### Transform evaluation set into numpy arrays

In [ ]:
#define source and destination folders
source = join(SOURCE, EVALUATION_SET_NAME)
destination = join(SOURCE, EVAL_FEATURES)

# create the eval_features directory if it does not exist
if not os.path.exists(destination):
    os.makedirs(destination)

# transform images in eval dataset (i.e. normalized pixel values and resized pictures) to numpy arrays (smaller files, useful if uploading the set to train a model in the cloud)
for f in os.listdir(source):
    if f.find('.png') != -1:
        img = Utils.get_preprocessed_img(join(source, f), IMAGE_SIZE)
        file_name = f[:f.find('.png')]

        np.savez_compressed(join(destination, file_name), features=img)
        retrieve = np.load(join(destination, file_name + '.npz'))['features']
        
        assert np.array_equal(img, retrieve)
        
        shutil.copyfile(join(source, file_name + '.gui'), join(destination, file_name + '.gui'))

In [ ]:
# make folder to store runtime files
if not os.path.exists('bin'):
    os.mkdir('bin')

### Create folder to store checkpoint files

In [ ]:
# make folder to store checkpoint files
if not os.path.exists(CHECKPOINT_DIR):
    os.mkdir(CHECKPOINT_DIR)

### Declare class of callbacks - used to save model after each epoch

In [ ]:
from tensorflow.keras.callbacks import Callback
from google.colab import files

class TrainingCallback(Callback):
    def on_train_begin(self, logs=None):
        print("Starting training...")
        # Load weights if there are any in the training folder
        if os.path.isfile(CHECKPOINT_ARCHIVE_PATH):
            print('Decompressing weights...')
            !unzip {CHECKPOINT_ARCHIVE_PATH}
            print('Loading weights...')
            self.model.load_weights(CHECKPOINT_PATH)
            !rm {CHECKPOINT_PATH}

        
    def on_epoch_begin(self, epoch, logs=None):
        if os.path.isfile(CHECKPOINT_PATH):
            print('Compressing weights checkpoint...')
            !zip -9 {CHECKPOINT_ARCHIVE_PATH} {CHECKPOINT_PATH}
            !rm {CHECKPOINT_PATH}
            # every 3 epochs download the weights
            if epoch%3==0:
                print('Downloading checkpoint...')
                files.download(CHECKPOINT_ARCHIVE_PATH)

### Declare magicode class

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, \
                         RepeatVector, LSTM, concatenate, \
                         Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras import *
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard


class magicode:
    def __init__(self, input_shape, output_size, output_path):
        self.model = None
        self.name = 'magicode'
        self.input_shape = input_shape
        self.output_size = output_size
        self.output_path = output_path

        image_model = Sequential()
        image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', input_shape=input_shape))
        image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu'))
        image_model.add(MaxPooling2D(pool_size=(2, 2)))
        image_model.add(Dropout(0.25))

        image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
        image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
        image_model.add(MaxPooling2D(pool_size=(2, 2)))
        image_model.add(Dropout(0.25))

        image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
        image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
        image_model.add(MaxPooling2D(pool_size=(2, 2)))
        image_model.add(Dropout(0.25))

        image_model.add(Flatten())
        image_model.add(Dense(1024, activation='relu'))
        image_model.add(Dropout(0.3))
        image_model.add(Dense(1024, activation='relu'))
        image_model.add(Dropout(0.3))

        image_model.add(RepeatVector(CONTEXT_LENGTH))

        visual_input = Input(shape=input_shape)
        encoded_image = image_model(visual_input)

        language_model = Sequential()
        language_model.add(LSTM(128, return_sequences=True, input_shape=(CONTEXT_LENGTH, output_size)))
        language_model.add(LSTM(128, return_sequences=True))

        textual_input = Input(shape=(CONTEXT_LENGTH, output_size))
        encoded_text = language_model(textual_input)

        decoder = concatenate([encoded_image, encoded_text])

        decoder = LSTM(512, return_sequences=True)(decoder)
        decoder = LSTM(512, return_sequences=False)(decoder)
        decoder = Dense(output_size, activation='softmax')(decoder)

        self.model = Model(inputs=[visual_input, textual_input], outputs=decoder)

        optimizer = RMSprop(lr=LEARNING_RATE, clipvalue=CLIPVALUE)
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    def fit_generator(self, generator, eval_generator, steps_per_epoch, validation_steps):
        # TensorBoard used to produce graphs - callback run every epoch
        tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
        # monitors the validation_loss value and when it stops decreasing the model will run for PATIENCE more times
        # if the value doesn't decrease then restore the weights from the run with the smallest val_loss value
        early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', patience=PATIENCE, restore_best_weights=True, verbose = 1)

        # monitors the validation_loss value and when it stops decreasing the model will run for PATIENCE more times
        # if the value doesn't decrease then lower the learning_rate by a factor of 0.1
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001, verbose=1)

        # Checkpoint callback to save the model's weights
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH, save_weights_only=True, verbose=1)

        self.model.fit(generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, validation_data=eval_generator, validation_steps=validation_steps, verbose=1, callbacks=[tensorboard, early_stopping, reduce_lr, cp_callback, TrainingCallback()])
        self.save()

    def predict(self, image, partial_caption):
        return self.model.predict([image, partial_caption], verbose=0)[0]

    def save(self):
        model_json = self.model.to_json()
        with open(join(self.output_path, self.name + '.json'), "w") as json_file:
            json_file.write(model_json)
        self.model.save_weights(join(self.output_path, self.name + '.h5'))

    def load(self, name=""):
        output_name = self.name if name == "" else name
        with open(join(self.output_path, output_name + '.json'), "r") as json_file:
            loaded_model_json = json_file.read()
        self.model = model_from_json(loaded_model_json)
        self.model.load_weights(join(self.output_path, output_name + '.h5'))

### Train the model using a generator (to avoid having to fit all the data in memory)

In [ ]:
import tensorflow as tf

np.random.seed(1234)

training_features = join(SOURCE, TRAINING_FEATURES)
eval_features = join(SOURCE, EVAL_FEATURES)
output_path = join('bin')

dataset = Dataset()
dataset.dataset_name = 'training'
dataset.load(training_features, generate_binary_sequences=True)
dataset.save_metadata(output_path)
dataset.voc.save(output_path)

gui_paths, img_paths = Dataset.load_paths_only(training_features)

input_shape = dataset.input_shape
output_size = dataset.output_size
steps_per_epoch = dataset.size / BATCH_SIZE
voc = Vocabulary()
voc.retrieve(output_path)

generator = Generator.data_generator(voc, gui_paths, img_paths, batch_size=BATCH_SIZE, generate_binary_sequences=True)

eval_dataset = Dataset()
eval_dataset.dataset_name = 'evaluation'
eval_dataset.load(eval_features, generate_binary_sequences=True)
eval_gui_paths, eval_img_paths = Dataset.load_paths_only(eval_features)
validation_steps = eval_dataset.size / BATCH_SIZE

eval_generator = Generator.data_generator(voc, eval_gui_paths, eval_img_paths, batch_size=BATCH_SIZE, generate_binary_sequences=True)

model = magicode(input_shape, output_size, output_path)

# NOTE: upload any saved zip checkpoint to the training folder to start training from where it left off

model.fit_generator(generator, eval_generator, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

In [ ]:
# Plot the model (Place after model = magicode... line in cell above)

# THIS RETURNS ERROR: ('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')

# tf.keras.utils.plot_model(
#     model,
#     to_file='model.png',
#     show_shapes=False,
#     show_dtype=False,
#     show_layer_names=True,
#     rankdir='LR',
#     expand_nested=False,
#     dpi=96,
# )

### Start tensorboard to visualise the training results

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

### Create the directories for storing screenshots to "decode" and the resulting code

In [ ]:
# create directory to store images to be "decoded"
if not os.path.exists('screenshots_to_convert'):
    os.mkdir('screenshots_to_convert')
# create directory to store HTML and GUI code generated by "decoding" images in screenshots folder 
if not os.path.exists('generated_code'):
    os.mkdir('generated_code')

### Generate the code for provided screenshots - add files to the screenshots folder

In [ ]:
trained_weights_path = 'bin'
trained_model_name = 'magicode'
input_path = 'screenshots_to_convert'
output_path = 'generated_code'

meta_dataset = np.load(join(trained_weights_path, 'meta_dataset.npy'), allow_pickle=True)
input_shape = meta_dataset[0]
output_size = meta_dataset[1]

model = magicode(input_shape, output_size, trained_weights_path)
model.load(trained_model_name)

sampler = Sampler(trained_weights_path, input_shape, output_size, CONTEXT_LENGTH)

for f in os.listdir(input_path):
    if f.find('.png') != -1:
        evaluation_img = Utils.get_preprocessed_img(join(input_path,f), IMAGE_SIZE)

        file_name = f[:f.find('.png')]

        result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
        print('Result greedy: {}'.format(result))

        with open(join(output_path, file_name + '.gui'), 'w') as out_f:
            out_f.write(result.replace(START_TOKEN, '').replace(END_TOKEN, ''))

### Declare compiler

In [ ]:
import json
import string
import random

class Compiler:
    def __init__(self, dsl_mapping_file_path):
        with open(dsl_mapping_file_path) as data_file:
            self.dsl_mapping = json.load(data_file)

        self.opening_tag = self.dsl_mapping['opening-tag']
        self.closing_tag = self.dsl_mapping['closing-tag']
        self.content_holder = self.opening_tag + self.closing_tag

        self.root = Node('body', None, self.content_holder)

    def compile(self, input_file_path, output_file_path, rendering_function=None):
        dsl_file = open(input_file_path)
        current_parent = self.root

        for token in dsl_file:
            token = token.replace(' ', '').replace('\n', '')

            if token.find(self.opening_tag) != -1:
                token = token.replace(self.opening_tag, '')

                element = Node(token, current_parent, self.content_holder)
                current_parent.add_child(element)
                current_parent = element
            elif token.find(self.closing_tag) != -1:
                current_parent = current_parent.parent
            else:
                tokens = token.split(',')
                for t in tokens:
                    element = Node(t, current_parent, self.content_holder)
                    current_parent.add_child(element)

        output_html = self.root.render(self.dsl_mapping, rendering_function=rendering_function)
        with open(output_file_path, 'w') as output_file:
            output_file.write(output_html)

class Node:
    def __init__(self, key, parent_node, content_holder):
        self.key = key
        self.parent = parent_node
        self.children = []
        self.content_holder = content_holder

    def add_child(self, child):
        self.children.append(child)

    def show(self):
        print(self.key)
        for child in self.children:
            child.show()

    def render(self, mapping, rendering_function=None):
        content = ''
        for child in self.children:
            content += child.render(mapping, rendering_function)

        value = mapping[self.key]
        if rendering_function is not None:
            value = rendering_function(self.key, value)

        if len(self.children) != 0:
            value = value.replace(self.content_holder, content)

        return value

class CompilerUtils:
    @staticmethod
    def get_random_text(length_text=10, space_number=1, with_upper_case=True):
        results = []
        while len(results) < length_text:
            char = random.choice(string.ascii_letters[:26])
            results.append(char)
        if with_upper_case:
            results[0] = results[0].upper()

        current_spaces = []
        while len(current_spaces) < space_number:
            space_pos = random.randint(2, length_text - 3)
            if space_pos in current_spaces:
                break
            results[space_pos] = " "
            if with_upper_case:
                results[space_pos + 1] = results[space_pos - 1].upper()

            current_spaces.append(space_pos)

        return ''.join(results)

### Compile the generated code

In [ ]:
FILL_WITH_RANDOM_TEXT = True
TEXT_PLACE_HOLDER = '[]'

dsl_path = join('compiler','assets','dsl-mapping.json')
compiler = Compiler(dsl_path)

def render_content_with_text(key, value):
    text_inputs = ['input-text', 'input-password']
    control_inputs = ['input-checkbox', 'input-radio']
    if FILL_WITH_RANDOM_TEXT:
        if key.find('btn') != -1:
            value = value.replace(TEXT_PLACE_HOLDER, CompilerUtils.get_random_text())
        elif key.find('title') != -1:
            value = value.replace(TEXT_PLACE_HOLDER, CompilerUtils.get_random_text(length_text=5, space_number=0))
        elif key.find('text') != -1:
            value = value.replace(TEXT_PLACE_HOLDER,
                                  CompilerUtils.get_random_text(length_text=56, space_number=7, with_upper_case=False))
        elif any(text_input in key for text_input in text_inputs):
            value = value.replace(TEXT_PLACE_HOLDER, CompilerUtils.get_random_text(length_text=30, space_number=0))
        elif any(control_input in key for control_input in control_inputs):
            value = value.replace(TEXT_PLACE_HOLDER, CompilerUtils.get_random_text(length_text=10, space_number=0))
    return value

path = 'generated_code'
generated_code_files = os.listdir(path)

for file in generated_code_files:
    file_uid = Path(file).stem
    input_file_path = join(path, file_uid + '.gui')
    output_file_path = join(path, file_uid + '.html')

    compiler.compile(input_file_path, output_file_path, rendering_function=render_content_with_text)